In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from polyline import Polyline

from curvegen import Curve2D
from egoaxis import rot_axis_90deg_cc

## Module

In [ ]:
# Note:
# To make it match with the previous measurement, init with the last point
# of the previous measurement and keep it fixed.
# Use the largest principal component vector, project points, keep last point
# fixed an rotate line around this point, so the first point matches the
# reference point from the last measurement.

def proj_points(pts, vec):
    """
    Projects points with shape (len(vec), N) along direction of vec (1d-array).
    """
    vec = np.asarray(vec)
    pts = np.atleast_2d(pts)
    
    unit = vec / np.linalg.norm(vec)
    proj_len = np.matmul(pts.T, vec)

    return proj_len * vec.reshape(len(vec), 1), proj_len

def poly_principal_curve(data, niter="auto"):
    """
    Computes a principal curve for the given data points (shape (2, n)).
    Source
    ------
    Kegl: Learning and Design of Principal Curves
        (https://www.lri.fr/~kegl/research/PDFs/KeKrLiZe00.pdf)
    """
    data = np.atleast_2d(data)
    if len(data.shape) != 2 or data.shape[0] != 2:
        raise ValueError("'data' must have shape (2, nPoints).")
    npoints = data.shape[1]
    
    # Center data mean at  (0, 0)
    mean = np.mean(data, axis=1).reshape(2, 1)
    data -= mean
    
    # Following the algorithm in chapter 3 of the paper
    
    # Initialization: Use first principal value of data as projection line d and
    # pick the line length L as the distance of min, max points projected on d
    # Get direction and value of largest eigenvalue from cov(data)
    cov = np.cov(data)
    eigvals, eigvecs = np.linalg.eigh(cov)
    eigvec_max = eigvecs[np.argmax(eigvals)]
    # Project along largest eigenvector and determine line length by finding
    # the outermost projected points along the projection line
    proj_vec, proj_len = proj_points(data, eigvec_max)
    idx = np.argsort(proj_len)
    # Keep the original order intact for the projection
    idx_min, idx_max = ((idx[0], idx[-1]) if
                        idx[0] < idx[-1] else (idx[-1], idx[0]))
    dmin, dmax = data[:, idx_min], data[:, idx_max]
    pl = Polyline([dmin, dmax])
    
    # Data radius used for scaling in the penalty term
    data_r2 = np.amax(np.linalg.norm(data, axis=0))**2
    
    def get_penalty(idx, pl):
        """ Return penalty termfor given vertex in polyline pl """
        pi = lambda i: data_r2 * np.cos(pl.get_angle_at_vertex(i))
        mu_plus = lambda i: data_r2 * np.cos(pl.get_angle_at_vertex(i))
        mu_minus = lambda i: data_r2 * np.cos(pl.get_angle_at_vertex(i))
                
        if idx == 0:
            return mu_plus(idx) + pi(idx + 1)
        if idx == 1:
            return mu_minus(idx) + pi(idx) + pi(idx + 1)
        if idx == pl.nvertices - 1:
            return pi(idx - 1) + pi(idx) + mu_plus(idx)
        if idx == pl.nvertices:
            return pi(idx - 1) + mu_minus(idx)
        else:
            return pi(idx - 1) + pi(idx) + pi(idx + 1)
        
    def converged(pl):
        """ Returns True, if given polyline reaches convergence criteria """
    
    # Iteration: Add vertex each iter until converged or max vertices reached
    # Empirical from paper, at least 2
    max_vertices = max(int(npoints**(1. / 3.)) + 1, 2)
    for i in range(max_vertices):
        # Vertex optimization:
        # Optimize each existing vertex while fixing the others unti the
        # polyline with k current vertices is locally closest to all points
        step_converged = False
        while not step_converged:
            for j, vert in pl.vertices:
                pass
            step_converged = True
    
        if converged(pl):
            break  # Break early if converged
    
    # Move back to actual mean
    pl.translate(mean)
    return pl

## Testplots

### Curve generator

In [ ]:
curve = Curve2D(curvature=-3., curve_rel_start=0.6)
x = np.linspace(0, 10, 50)
y, dy = curve(x)
x_sam = np.linspace(1, 10, 20)
x_sam, y_sam = curve.sample(x_sam, stddev=0.4, seed=2)

fig, ax = plt.subplots(1, 1)
ax.plot(x, y)
ax.plot(x_sam, y_sam, marker="o", ls="-")

ax.set_xlim(0, 1.1 * max(np.amax(x_sam), np.amax(x)))
ax.set_ylim(-5, 5)

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.plot(0, 0, marker=10, ms=10, ls="none", c="k")

ax = rot_axis_90deg_cc(ax)
ax.grid()
ax.axvline(0, 0, 1, c="k", lw=1, zorder=1)
ax.set_axisbelow(True)

plt.show()

### Distance to polyline

In [ ]:
pl = Polyline([[1, 3], [3, 5], [6, 2], [2, 2]])
pts = [[1, 2], [1.5, 5.5], [3, 6], [3.5, 3.5], [5.5, 3.5], [4, 1], [2, 2.5]]
proj_vecs, dists, proj_to = pl.get_dist_to_line(pts)
pts = np.array(pts)

In [ ]:
plt.plot(pl.xcoords, pl.ycoords, ls="-", marker="o", c="C0")
plt.plot(pts[:, 0], pts[:, 1], marker="x", lw=4, c="k", ls="none")

for pti, pvi in zip(pts, proj_vecs):
    plt.arrow(pti[0], pti[1], pvi[0], pvi[1])
    
print(proj_to)
print(dists)

plt.xlim(0, None)
plt.ylim(0, None)
plt.gca().set_aspect("equal")
plt.show()

### Project on eigenvector

In [ ]:
curve = Curve2D(curvature=-4., curve_rel_start=0.3)
x = np.linspace(0, 10, 50)
y, dy = curve(x)
x_sam = np.linspace(1, 10, 20)
x_sam, y_sam = curve.sample(x_sam, stddev=0.4, seed=2)

data = np.asarray([x_sam, y_sam])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

# Truth and sampled points
ax.plot(x, y)
ax.plot(x_sam, y_sam, marker="o", ls="-")

ax.set_xlim(0, 1.1 * max(np.amax(x_sam), np.amax(x)))
ax.set_ylim(-5, 5)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.plot(0, 0, marker=10, ms=10, ls="none", c="k")

# Principal value line
eigvec, _, _ = get_max_eigvec(data)
mean = np.mean(data, axis=1)
mx = np.r_[mean[0] - 10 * eigvec[0], mean[0] + 10 * eigvec[0]]
my = np.r_[mean[1] - 10 * eigvec[1], mean[1] + 10 * eigvec[1]]
ax.plot(mean[0], mean[1], ls="none", marker="x", ms=10, c="C3", mew=3, zorder=1)
ax.plot(mx, my, ls="--", c="C3", lw=2)

# Projected points
mean = np.mean(data, axis=1).reshape(2, 1)
proj_vec, proj_len = proj_points(data - mean, eigvec)
proj_vec += mean
for proj, pt in zip(proj_vec.T, data.T):
    dx = [proj[0], pt[0]]
    dy = [proj[1], pt[1]]
    ax.plot(dx, dy, lw=1, c="k")

# ax = rot_axis_90deg_cc(ax)
ax.grid()
ax.axvline(0, 0, 1, c="k", lw=1, zorder=1)
ax.axhline(0, 0, 1, c="k", lw=1, zorder=1)
ax.set_axisbelow(True)
ax.set_aspect("equal")

plt.show()